In [1]:
import smtplib, subprocess, os, boto3, cv2
import pywhatkit as kt
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
import time

# Defining some functions

In [2]:
def face_extractor(img):
    #Function detects faces and returns the cropped face
    #If no face detected, it returns the input image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)    
    if faces is ():
        return None
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

def send_mail(sender_email_id,sender_email_id_password,Mail_you_need_to_send,receiver_email_id):
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()#start TLS for security
    s.login(sender_email_id, sender_email_id_password)#Authentication
    mail = Mail_you_need_to_send#mail to be sent
    s.sendmail(sender_email_id, receiver_email_id, mail)#sending the mail
    s.quit()#terminating the session
    print("Mail is sent")
    
def send_whatsapp(p_num,whatsapp_msg):
    print("Let's Automate Whatsapp!")
    #p_num ->the target phone number goes here!'
    minute = pd.Timestamp.now().minute + 1
    hour = pd.Timestamp.now().hour
    kt.sendwhatmsg(p_num, whatsapp_msg, hour, minute)
    
def launch_ec2_create_and_attach_ebs():
    #launching ec2 instance
    os.system("aws ec2 run-instances --image-id ami-011c99152163a87ae --instance-type t2.micro --count 1 --subnet-id subnet-2eeeb462 --security-group-ids sg-251cdc59 --key-name \"Anamika Mitee\" ")
    time.sleep(10)
    
    ec2 = boto3.resource('ec2')
    for instance in ec2.instances.all():
        InstanceId =   "{0}".format(instance.id)
    InstanceId
    #creating 5 GB volume
    Volumeid = subprocess.getoutput('aws ec2 create-volume --availability-zone ap-south-1b --size 5 --query \"VolumeId\" --output text')
    Volumeid
    time.sleep(10)
    
    #attaching EBS volume to that instance
    to_attach = f'aws ec2 attach-volume --device /dev/sdf --instance-id {InstanceId} --volume-id {Volumeid}'
    to_attach
    os.system(to_attach)
    print("EC2 instance is launched, 5GB volume is created and attached to it!!!!! ")

# Creating training dataset

In [3]:
Names = []

In [7]:
ID = input("Enter the id of user: ")
Name = input("Enter the name of user: ")

Names.append(Name)
print(Names)

Enter the id of user: 2
Enter the name of user: CaptainAmerica
['Anamika', 'CaptainAmerica']


In [8]:
#Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#Initialize Webcam
cap = cv2.VideoCapture(1)
count = 0

# Collect samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = "./dataset/" + Name + "." + ID +"."+ str(count) + ".jpg"
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Cropped img', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found


# training model

In [9]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join

def Image_and_id(path):
    image_paths = [os.path.join(path,f) for f in os.listdir(path)]
    facesamples, Ids = [], []
    
    for imagepath in image_paths:
        image = cv2.imread(imagepath, cv2.IMREAD_GRAYSCALE)
        imagenp = np.asarray(image,dtype=np.uint8)
        Id = int(os.path.split(imagepath)[-1].split(".")[1])
        face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_classifier.detectMultiScale(imagenp, 1.3, 5)
        for (x,y,w,h) in faces:
            facesamples.append(imagenp[y:y+h, x:x+w])
            Ids.append(Id)
    return facesamples , Ids



model  = cv2.face.LBPHFaceRecognizer_create()
faces , Id = Image_and_id("./dataset/")
#Let's train our model 
model.train(faces, np.array(Id))
print("Model trained sucessefully")
model.save("./Trained_model.yml")

Model trained sucessefully


# running my facial recognization

In [11]:
#face recognition
import cv2
import numpy as np
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

launched = False
sent = False

# Open Webcam
cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        results = model.predict(face)
        id = results[0]
       
        if results[1] < 500:
            conf = int(100*(1- results[1]/400))
            st = str(conf) + '% confidence of ' + Names[id-1]
            cv2.putText(image, st, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face_Recognition', image ) 
            
        if conf > 80 and id == 2 and launched == False:
            cv2.imshow('Face_Recognition', image )  
            print("launched",conf)
             #launching ec2 instance, creating 5GB volume and attaching to it
            launch_ec2_create_and_attach_ebs()
            launched = True
            break
            
            
        elif conf > 80 and id == 1 and sent == False:
            cv2.imshow('Face_Recognition', image )  
            #Send Mail
            sender_email_id = "aaviagalav024@gmail.com"
            sender_email_id_password = "qtzlqwogugfuprps"
            Mail_you_need_to_send   = "This is your face"
            receiver_email_id   = "aaviagalav024@gmail.com"
            send_mail(sender_email_id,sender_email_id_password,Mail_you_need_to_send,receiver_email_id)
            
            #Send msg to a whatsapp number
            p_num = "+916264964403"
            whatsapp_msg = "Hey Anamika! This is your face!!!"
            send_whatsapp(p_num,whatsapp_msg)
            sent = True
            break
            
        else:
            pass

            
    except:
        cv2.putText(image, "No Face Found", (50, 50) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face_Recognition', image )
        pass
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break    

launched 81
EC2 instance is launched, 5GB volume is created and attached to it!!!!! 


In [ ]:
cap.release()
cv2.destroyAllWindows()
